<a href="https://colab.research.google.com/github/qasimzee/isic2024/blob/main/isic_2024_challenge_table_only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-learn==1.2.2
!pip install numpy==1.26.4

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 20.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "/content/drive/MyDrive/kaggle/isic-2024-data"

cnn.pkl			 sample_submission.csv	test-metadata.csv	  train-metadata.csv
logistic_regression.pkl  scaler.pkl		test-metadata.csv.gsheet  train-metadata-v1.csv
random_forest_model.pkl  test-image.hdf5	train-image.hdf5


In [ ]:
METADATA_FILE = "/content/drive/MyDrive/kaggle/isic-2024-data/train-metadata.csv"
metadata_df = pd.read_csv(METADATA_FILE)


<ipython-input-4-6d834c4c6865>:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata_df = pd.read_csv(METADATA_FILE)


In [ ]:
train_df, test_df = train_test_split(metadata_df, test_size=0.2, random_state=42)

train_df


isic_id  target  patient_id  age_approx     sex  \
18085   ISIC_0517312       0  IP_5299455        60.0  female   
171181  ISIC_4318651       0  IP_3923321        70.0  female   
284095  ISIC_7110993       0  IP_1544679        55.0    male   
354900  ISIC_8861121       0  IP_1116526        80.0    male   
77679   ISIC_1994958       0  IP_5067302        70.0    male   
...              ...     ...         ...         ...     ...   
259178  ISIC_6502755       0  IP_7965642        60.0    male   
365838  ISIC_9129069       0  IP_2889176        45.0  female   
131932  ISIC_3342513       0  IP_9577633        50.0    male   
146867  ISIC_3720179       0  IP_3751225        75.0    male   
121958  ISIC_3097113       0  IP_2573038        50.0    male   

       anatom_site_general  clin_size_long_diam_mm          image_type  \
18085      posterior torso                    2.86  TBP tile: close-up   
171181      anterior torso                    2.61  TBP tile: close-up   
284095     posterior torso                    2.79  TBP tile: close-up   
354900     lower extremity                    6.68  TBP tile: close-up   
77679      posterior torso                    3.63  TBP tile: close-up   
...                    ...                     ...                 ...   
259178     upper extremity                    3.94  TBP tile: close-up   
365838     posterior torso                    2.74  TBP tile: close-up   
131932     lower extremity                    3.26  TBP tile: close-up   
146867      anterior torso                    2.60  TBP tile: close-up   
121958      anterior torso                    3.87  TBP tile: close-up   

       tbp_tile_type   tbp_lv_A  ...  lesion_id  iddx_full  iddx_1  iddx_2  \
18085         3D: XP  20.465960  ...        NaN     Benign  Benign     NaN   
171181     3D: white  19.067292  ...        NaN     Benign  Benign     NaN   
284095        3D: XP  24.721678  ...        NaN     Benign  Benign     NaN   
354900        3D: XP  16.342123  ...        NaN     Benign  Benign     NaN   
77679         3D: XP  20.388077  ...        NaN     Benign  Benign     NaN   
...              ...        ...  ...        ...        ...     ...     ...   
259178        3D: XP  14.689290  ...        NaN     Benign  Benign     NaN   
365838        3D: XP  24.387236  ...        NaN     Benign  Benign     NaN   
131932        3D: XP  18.711190  ...        NaN     Benign  Benign     NaN   
146867        3D: XP  16.110687  ...        NaN     Benign  Benign     NaN   
121958     3D: white  23.701634  ...        NaN     Benign  Benign     NaN   

        iddx_3  iddx_4  iddx_5  mel_mitotic_index  mel_thick_mm  \
18085      NaN     NaN     NaN                NaN           NaN   
171181     NaN     NaN     NaN                NaN           NaN   
284095     NaN     NaN     NaN                NaN           NaN   
354900     NaN     NaN     NaN                NaN           NaN   
77679      NaN     NaN     NaN                NaN           NaN   
...        ...     ...     ...                ...           ...   
259178     NaN     NaN     NaN                NaN           NaN   
365838     NaN     NaN     NaN                NaN           NaN   
131932     NaN     NaN     NaN                NaN           NaN   
146867     NaN     NaN     NaN                NaN           NaN   
121958     NaN     NaN     NaN                NaN           NaN   

        tbp_lv_dnn_lesion_confidence  
18085                      99.739540  
171181                     99.990857  
284095                    100.000000  
354900                     95.420188  
77679                      99.998164  
...                              ...  
259178                     94.829340  
365838                    100.000000  
131932                     99.980790  
146867                     99.995995  
121958                     96.398944  

[320847 rows x 55 columns]

In [ ]:
dropped_columns = ["isic_id", "target", "patient_id", "lesion_id", "iddx_full", "iddx_1", "iddx_2", "iddx_3", "iddx_4", "iddx_5", "mel_mitotic_index", "mel_thick_mm", "tbp_lv_dnn_lesion_confidence"]
X = train_df.drop(columns=dropped_columns)
Y = train_df["target"]

# Replace NaN values
X['sex'] = X['sex'].replace(pd.NA, 'unknown')
X['anatom_site_general'] = X['anatom_site_general'].replace(pd.NA, 'unknown')



X.fillna(0, inplace=True)

In [ ]:
# Identify columns with mixed types

for column in X.columns:
  if X[column].apply(type).nunique() > 1:
    print(f"{column}: {X[column].apply(type).unique()}")


In [ ]:
# Identify categorical features
categorical_features = X.select_dtypes(include=['object']).columns

# Create a ColumnTransformer with OneHotEncoder
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features)],
    remainder='passthrough')  # Pass through numerical columns unchanged


# Create a pipeline that first transforms the data, then fits the model
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('scaler', StandardScaler()),
                           ('model', LogisticRegression())])

# Fit the pipeline on the training data
pipeline.fit(X, Y)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  Index(['sex', 'anatom_site_general', 'image_type', 'tbp_tile_type',
       'tbp_lv_location', 'tbp_lv_location_simple', 'attribution',
       'copyright_license'],
      dtype='object'))])),
                ('scaler', StandardScaler()), ('model', LogisticRegression())])

In [ ]:
test_df['sex'] = test_df['sex'].replace(pd.NA, 'unknown')
test_df['anatom_site_general'] = test_df['anatom_site_general'].replace(pd.NA, 'unknown')
test_df['lesion_id'] = test_df['lesion_id'].replace(pd.NA, 'unknown')
test_df['iddx_2'] = test_df['iddx_2'].replace(pd.NA, 'unknown')
test_df['iddx_3'] = test_df['iddx_3'].replace(pd.NA, 'unknown')
test_df['iddx_4'] = test_df['iddx_4'].replace(pd.NA, 'unknown')
test_df['mel_mitotic_index'] = test_df['mel_mitotic_index'].replace(pd.NA, 'unknown')

test_df.fillna(0, inplace=True)
# Identify columns with mixed types
for column in test_df.columns:
  if test_df[column].apply(type).nunique() > 1:
    print(f"{column}: {test_df[column].unique()}")


test_isic_ids = test_df['isic_id']
test_df = test_df.drop(columns=dropped_columns)


In [ ]:
# Make predictions on the test data
y_pred_proba = pipeline.predict_proba(test_df)[:, 1]  # Probability of positive class (e.g., malignant)

# Create a DataFrame to map predictions back to 'isic_id'
predictions_df = pd.DataFrame({
    'isic_id': test_isic_ids,
    'target': y_pred_proba
})

# Print or save the predictions
# Optionally save to CSV
predictions_df #.to_csv('submission.csv', index=False)
predictions_df['target'].min()
# # Create the submission DataFrame, using the saved 'isic_id'
# submission_df = pd.DataFrame({
#     'isic_id': test_isic_ids,
#     'target': test_predictions
# })

# # # Save to CSV
# submission_df #.to_csv('submission.csv', index=False)




9.499789169965982e-09

In [ ]:
import pickle
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression

# Assume you have trained your pipeline as described before

# Save the entire pipeline to a pickle file
with open('/content/drive/MyDrive/kaggle/isic-2024-data/logistic_regression.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

print("Model pipeline saved successfully!")


Model pipeline saved successfully!
